# RNN in Pytorch
## example : Yelp classifier

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [25]:
import os
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchtext import data as ttdata
from torchtext.data import Dataset, Example, Field
from torchtext.data import Iterator, BucketIterator
import spacy

from TextDataloader import TextData

In [3]:
if torch.cuda.device_count()>1:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
elif torch.cuda.device_count()>0:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Dataload
* reference : Natural language processing with PYTORCH
1. load data file into a dataframe
2. change text into right form using tokenizer or corpus
3. change modified text into vectors

In [35]:
lstm = nn.LSTM(input_size=size_in, 
        hidden_size=size_hidden, 
        dropout=dropout,
        num_layers= n_layers,
        batch_first=batch_first,
        bidirectional=bidirectional)

#########################3

n_hiddens = [size_hidden, ...]
mlp = get_MLP
get_RNN(10, 20, 4)

LSTM(10, 20, num_layers=4, batch_first=True, dropout=0.1)

In [16]:
class RNN_classifier(Net):
    def __init__(self, model=None, loss=None, 
                 optimizer=None):
        pass
    
    def run_batch(self, i_batch, data):
        pass
    
    def run_train(self, n_epoch, data):
        pass
    
    def run_eval(self, data):
        pass
    
    
    
    

In [16]:
"""
data shape
rating, review, split
negative, "sentence", train
positive, "sentence", train
"""
f1_s = "/home/bwlee/data/yelp_review_polarity_csv/reviews_with_splits_full.csv"

#df1 = pd.read_csv(f1_s, header=0)
#df1 = pd.read_csv(f1_s, header=0, nrows=1000)
df = pd.read_csv(f1_s, header=0, nrows=10000, skiprows=lambda x: x%5>0)
label0 = []
for i in df.index: # change this for getting info based on previous day market move
    if df['rating'].iloc[i] == 'positive':
        label0.append(1)
    else:
        label0.append(0)
df['rating'] = label0

In [19]:
train_df = df[df['split']=='train'][['rating', 'review']]
val_df = df[df['split']=='val'][['rating', 'review']]
test_df = df[df['split']=='test'][['rating', 'review']]
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [20]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# vocabulary class
#voca1 = Vocab(df1, glove1)
#dataset1 = ReviewDataset(df1, voca1)

In [21]:
TEXT = data.Field(sequential=True, use_vocab=True,
                 tokenize='spacy', lower=True,
                 batch_first=True, fix_length=100,
                 init_token='<SOS>', eos_token='<EOS>'
                 )

LABEL = data.Field(sequential=False, use_vocab=False,
                  batch_first=False, is_target=True)

In [22]:
fields0 = (('rating', LABEL), ('review', TEXT))
train_data = [ Example.fromlist(
    train_df.iloc[ii].values.tolist(),
                              fields0)
              for ii in range(len(train_df)) ]
train_data = Dataset(train_data, fields=fields0)
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [23]:
iter = Iterator(train_data, batch_size=10)